In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
df_atraso = pd.read_csv(r'BD Tratado\atraso.csv',sep = ';')
df_atraso = df_atraso[['MATRÍCULA', 'DATA', 'ATRASO']]
df_atraso = df_atraso.sort_values(['MATRÍCULA', 'DATA'])
df_atraso['DATA'] = pd.to_datetime(df_atraso['DATA'])
df_atraso = df_atraso[df_atraso['ATRASO'] != 0]
df_atraso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97402 entries, 5422 to 761014
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   MATRÍCULA  97402 non-null  int64         
 1   DATA       97402 non-null  datetime64[ns]
 2   ATRASO     97402 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.0 MB


In [3]:
df_carga = pd.read_csv(r'BD Tratado\carga.csv',sep=';')
df_carga = df_carga.sort_values(['MATRÍCULA', 'DATA'])
df_carga['DATA FIM'] = np.where(df_carga['MATRÍCULA'].shift(-1) == df_carga['MATRÍCULA'], df_carga['DATA'].shift(-1), '2021-04-30')
df_carga['DATA FIM'] = pd.to_datetime(df_carga['DATA FIM'])
df_carga['DATA'] = pd.to_datetime(df_carga['DATA'])
df_carga.head()

,MATRÍCULA,DATA,CRG. HOR. DIA,DATA FIM
16225,3108,2020-02-15,NaN,2020-06-10
12967,3108,2020-06-10,08:30,2021-01-29
3447,3108,2021-01-29,08:48,2021-04-30
16923,3143,2020-01-24,08:00,2020-05-21
13609,3143,2020-05-21,NaN,2021-01-29


In [4]:
df_carga.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 16225 to 2685
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MATRÍCULA      17593 non-null  int64         
 1   DATA           17593 non-null  datetime64[ns]
 2   CRG. HOR. DIA  13799 non-null  object        
 3   DATA FIM       17593 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 687.2+ KB


In [5]:
df_jornada = pd.read_csv(r'BD Tratado\horarios.csv', sep = ';')
df_jornada.drop(['Unnamed: 0', 'HORÁRIO'], axis=1, inplace=True)
df_jornada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   MATRÍCULA        164 non-null    int64
 1   Jornada - 5x2    164 non-null    int64
 2   Jornada - 6x1    164 non-null    int64
 3   Jornada - 12x36  164 non-null    int64
 4   Jornada - movel  164 non-null    int64
 5   Jornada - fixo   164 non-null    int64
dtypes: int64(6)
memory usage: 7.8 KB


In [6]:
df_jornada.head()

,MATRÍCULA,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,205827,0,1,0,1,0
1,90293,1,0,0,0,1
2,200400,1,0,0,0,1
3,200570,1,0,0,0,1
4,200981,0,1,0,1,0


In [7]:
df_noturno = pd.read_csv(r'BD Tratado\noturno.csv',sep=';')
df_noturno = df_noturno.sort_values(['MATRÍCULA', 'DATA'])
df_noturno['DATA FIM'] = np.where(df_noturno['MATRÍCULA'].shift(-1) == df_noturno['MATRÍCULA'], df_noturno['DATA'].shift(-1), '2021-04-30')
df_noturno['DATA FIM'] = pd.to_datetime(df_noturno['DATA FIM'])
df_noturno['DATA'] = pd.to_datetime(df_noturno['DATA'])
df_noturno.head(10)

,MATRÍCULA,DATA,NOTURNO,DATA FIM
11163,3108,2020-02-13,0,2020-02-14
11127,3108,2020-02-14,0,2020-06-10
9616,3108,2020-06-10,0,2020-06-30
9265,3108,2020-06-30,0,2020-07-10
9097,3108,2020-07-10,0,2020-07-30
8760,3108,2020-07-30,0,2020-08-26
8263,3108,2020-08-26,0,2021-01-29
3167,3108,2021-01-29,0,2021-04-30
11398,3143,2020-01-24,0,2021-01-29
2775,3143,2021-01-29,0,2021-04-30


In [8]:
df = df_atraso.merge(df_jornada, how = 'left', on = 'MATRÍCULA')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo
0,3108,2020-01-03,1.0,NaN,NaN,NaN,NaN,NaN
1,3108,2020-01-10,72.0,NaN,NaN,NaN,NaN,NaN
2,3108,2020-01-20,46.0,NaN,NaN,NaN,NaN,NaN
3,3108,2020-01-21,47.0,NaN,NaN,NaN,NaN,NaN
4,3108,2020-01-22,32.0,NaN,NaN,NaN,NaN,NaN


In [9]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_carga.to_sql('df_carga', conn, index=False)

qry = '''
    select  distinct
        t1.*,
        t2.[CRG. HOR. DIA]
        
    from
        df as t1
        left join df_carga as t2
            on t1.DATA between t2.DATA and t2.[DATA FIM] 
            and t1.MATRÍCULA = t2.MATRÍCULA
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98310 entries, 0 to 98309
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MATRÍCULA        98310 non-null  int64  
 1   DATA             98310 non-null  object 
 2   ATRASO           98310 non-null  float64
 3   Jornada - 5x2    2726 non-null   float64
 4   Jornada - 6x1    2726 non-null   float64
 5   Jornada - 12x36  2726 non-null   float64
 6   Jornada - movel  2726 non-null   float64
 7   Jornada - fixo   2726 non-null   float64
 8   CRG. HOR. DIA    30927 non-null  object 
dtypes: float64(6), int64(1), object(2)
memory usage: 6.8+ MB


In [11]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_noturno.to_sql('df_noturno', conn, index=False)

qry = '''
    select  distinct
        t1.*,
        t2.[NOTURNO]
        
    from
        df as t1
        left join df_noturno as t2
            on t1.DATA between t2.DATA and t2.[DATA FIM] 
            and t1.MATRÍCULA = t2.MATRÍCULA
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98771 entries, 0 to 98770
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MATRÍCULA        98771 non-null  int64  
 1   DATA             98771 non-null  object 
 2   ATRASO           98771 non-null  float64
 3   Jornada - 5x2    2742 non-null   float64
 4   Jornada - 6x1    2742 non-null   float64
 5   Jornada - 12x36  2742 non-null   float64
 6   Jornada - movel  2742 non-null   float64
 7   Jornada - fixo   2742 non-null   float64
 8   CRG. HOR. DIA    31251 non-null  object 
 9   NOTURNO          49754 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 7.5+ MB


In [13]:
#analisando um caso
df[df['MATRÍCULA'] == 3108]

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN
5,3108,2020-01-23 00:00:00,36.0,NaN,NaN,NaN,NaN,NaN,None,NaN
6,3108,2020-01-24 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN
7,3108,2020-02-03 00:00:00,314.0,NaN,NaN,NaN,NaN,NaN,None,NaN
8,3108,2020-02-10 00:00:00,150.0,NaN,NaN,NaN,NaN,NaN,None,NaN
9,3108,2020-02-14 00:00:00,170.0,NaN,NaN,NaN,NaN,NaN,None,0.0


In [14]:
df_transporte = pd.read_csv(r'BD Tratado\Transp.csv',sep=';')
df_transporte['DATA FIM'] = pd.to_datetime(df_transporte['DATA FIM'])
df_transporte['DATA'] = pd.to_datetime(df_transporte['DATA'])
df_transporte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5918 entries, 0 to 5917
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MATRÍCULA         5918 non-null   int64         
 1   NÚM. VIAGENS DIA  5918 non-null   int64         
 2   VALOR POR VIAGEM  5918 non-null   float64       
 3   DATA              5918 non-null   datetime64[ns]
 4   DATA FIM          5918 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 231.3 KB


In [15]:
#Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
df_transporte.to_sql('df_transporte', conn, index=False)

qry = '''
    select distinct
        t1.*,
        t2.[NÚM. VIAGENS DIA],
        t2.[VALOR POR VIAGEM]
        
    from
        df as t1
        left join df_transporte as t2
            on t1.DATA between t2.DATA and t2.[DATA FIM] 
            and t1.MATRÍCULA = t2.MATRÍCULA
        '''
df = pd.read_sql_query(qry, conn)
df.head()

C:\Users\gabri\anaconda3\envs\faculdade\lib\site-packages\pandas\core\generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO,NÚM. VIAGENS DIA,VALOR POR VIAGEM
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MATRÍCULA         98855 non-null  int64  
 1   DATA              98855 non-null  object 
 2   ATRASO            98855 non-null  float64
 3   Jornada - 5x2     2742 non-null   float64
 4   Jornada - 6x1     2742 non-null   float64
 5   Jornada - 12x36   2742 non-null   float64
 6   Jornada - movel   2742 non-null   float64
 7   Jornada - fixo    2742 non-null   float64
 8   CRG. HOR. DIA     31265 non-null  object 
 9   NOTURNO           49783 non-null  float64
 10  NÚM. VIAGENS DIA  74167 non-null  float64
 11  VALOR POR VIAGEM  74167 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 9.1+ MB


In [17]:
df_func = pd.read_csv(r'BD Tratado\analiseFunc.csv',sep=';')
df_func.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5463 entries, 0 to 5462
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MATRÍCULA      5463 non-null   int64  
 1   Tempo de casa  5463 non-null   float64
 2   SALÁRIO        5463 non-null   float64
 3   SO             3628 non-null   float64
 4   SX             3628 non-null   float64
dtypes: float64(4), int64(1)
memory usage: 213.5 KB


In [18]:
df = df.merge(df_func, on = ['MATRÍCULA'],how='left')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO,NÚM. VIAGENS DIA,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN,4.0,5.65,195.385271,2184.57,3.0,0.0


In [19]:
df_demitidos = pd.read_csv(r'BD Tratado\demiticos.csv',sep=';')
df_demitidos['ADMISSÃO'] = pd.to_datetime(df_demitidos['ADMISSÃO'])
df_demitidos['DEMISSÃO'] = pd.to_datetime(df_demitidos['DEMISSÃO'])
df_demitidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3310 entries, 0 to 3309
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   NC Funcionário    3310 non-null   int64         
 1   ADMISSÃO          3310 non-null   datetime64[ns]
 2   CARGO             3310 non-null   object        
 3   DEMISSÃO          3310 non-null   datetime64[ns]
 4   Tipo de Rescisão  3306 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(2)
memory usage: 129.4+ KB


In [20]:
df = df.merge(df_demitidos, how = 'left', left_on='MATRÍCULA', right_on='NC Funcionário')
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,NC Funcionário,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaN,NaT,NaN,NaT,NaN


In [21]:
df['Ativo'] = np.where(df['NC Funcionário']>0,0,1)
df = df.drop(['NC Funcionário'],axis=1)
df.head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão,Ativo
0,3108,2020-01-03 00:00:00,1.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
1,3108,2020-01-10 00:00:00,72.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
2,3108,2020-01-20 00:00:00,46.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
3,3108,2020-01-21 00:00:00,47.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1
4,3108,2020-01-22 00:00:00,32.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,5.65,195.385271,2184.57,3.0,0.0,NaT,NaN,NaT,NaN,1


In [22]:
df[df['Ativo']==0].head()

,MATRÍCULA,DATA,ATRASO,Jornada - 5x2,Jornada - 6x1,Jornada - 12x36,Jornada - movel,Jornada - fixo,CRG. HOR. DIA,NOTURNO,...,VALOR POR VIAGEM,Tempo de casa,SALÁRIO,SO,SX,ADMISSÃO,CARGO,DEMISSÃO,Tipo de Rescisão,Ativo
69681,231495,2020-01-02 00:00:00,387.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69682,231495,2020-01-03 00:00:00,390.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69683,231495,2020-01-04 00:00:00,21.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69684,231495,2020-01-06 00:00:00,390.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0
69685,231495,2020-01-07 00:00:00,593.0,NaN,NaN,NaN,NaN,NaN,None,NaN,...,4.5,22.075757,1319.78,7.0,0.0,2019-06-21,Auxiliar de Limpeza B,2020-09-15,Demissão,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98855 entries, 0 to 98854
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MATRÍCULA         98855 non-null  int64         
 1   DATA              98855 non-null  object        
 2   ATRASO            98855 non-null  float64       
 3   Jornada - 5x2     2742 non-null   float64       
 4   Jornada - 6x1     2742 non-null   float64       
 5   Jornada - 12x36   2742 non-null   float64       
 6   Jornada - movel   2742 non-null   float64       
 7   Jornada - fixo    2742 non-null   float64       
 8   CRG. HOR. DIA     31265 non-null  object        
 9   NOTURNO           49783 non-null  float64       
 10  NÚM. VIAGENS DIA  74167 non-null  float64       
 11  VALOR POR VIAGEM  74167 non-null  float64       
 12  Tempo de casa     98855 non-null  float64       
 13  SALÁRIO           98855 non-null  float64       
 14  SO                8845

In [24]:
df.to_csv(r'BD Tratado\baseFim.csv',sep=';',index=False)